# SVM

## 0528
차원축소를 한 뒤 SVM을 적용해보자(이진분류)

#### valid로 검증해보기
- training/validation/Testing을 70:15:15로 나눈다.
- validation set으로 최상의 성능을 내는 옵션을 선택한다.
- validation set으로 선택된 것을 test set에 사용한다.


# 추가적으로 확인해봐야할것(다음시간에..)

- overfitting등의 문제에 대해 더 찾아보자
- PCA를 이용해서 주성분분석을 하여 선형차원축소를 해보자

In [1]:
!pip install xgboost

    100% |████████████████████████████████| 114.0MB 256kB/s eta 0:00:01
typing-extensions 3.7.4.1 has requirement typing>=3.7.4; python_version < "3.5", but you'll have typing 3.6.4 which is incompatible.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install --upgrade pip

    100% |████████████████████████████████| 1.5MB 17.2MB/s ta 0:00:01
typing-extensions 3.7.4.1 has requirement typing>=3.7.4; python_version < "3.5", but you'll have typing 3.6.4 which is incompatible.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1
You are using pip version 20.1.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# Data manipulation
import numpy as np
import pandas as pd

# Data pre-processing
from sklearn.preprocessing import StandardScaler as ss

# Dimensionality reduction
from sklearn.decomposition import PCA

#  Data splitting and model parameter search
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling modules
from xgboost.sklearn import XGBClassifier

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance

In [4]:
data=pd.read_csv("./data/binary_winequalityN.csv")
#dataw=pd.read_csv("./data/1_white.csv")
#datar=pd.read_csv("./data/1_red.csv")

data=data.dropna(axis=0)  ##없애버림, 평균값 넣으려면 axis=0대신에 inplate=True 사용
#dataw=dataw.dropna(axis=0)

print "wine 데이터셋의 크기:" ,data.shape
print "\nwine 데이터셋의 키:", data.keys()

wine 데이터셋의 크기: (6463, 14)

wine 데이터셋의 키: Index([u'type', u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol',
       u'quality', u'binaryQuality'],
      dtype='object')


데이터셋의 키
['type', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6463 entries, 0 to 6496
Data columns (total 14 columns):
type                    6463 non-null object
fixed acidity           6463 non-null float64
volatile acidity        6463 non-null float64
citric acid             6463 non-null float64
residual sugar          6463 non-null float64
chlorides               6463 non-null float64
free sulfur dioxide     6463 non-null float64
total sulfur dioxide    6463 non-null float64
density                 6463 non-null float64
pH                      6463 non-null float64
sulphates               6463 non-null float64
alcohol                 6463 non-null float64
quality                 6463 non-null int64
binaryQuality           6463 non-null int64
dtypes: float64(11), int64(2), object(1)
memory usage: 757.4+ KB


# 훈련 데이터와 테스트 데이터

In [6]:
#x_train에 적재하기 위한 2차원 배열의 특성 dataset을 만든다.
#6463*11(?) 사이즈

#실험의 편의성을 위해 data마다 이름을 붙였다.
f1=data['fixed acidity']
f2=data['volatile acidity']
f3=data['citric acid']
f4=data['residual sugar']
f5=data['chlorides']
f6=data['free sulfur dioxide']
f7=data['total sulfur dioxide']
f8=data['density']
f9=data['pH']
f10=data['sulphates']
f11=data['alcohol']


c_data=np.c_[f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11]

---

### 어떤 데이터를 적재?

#### 원본 

In [88]:
## 원본
'''
X_train, X_test, y_train, y_test = train_test_split(c_data, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
X_train, X_test, y_train, y_test = train_test_split(c_data, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)

print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_val 크기: ", X_val.shape
print "y_val 크기: ", y_val.shape
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (4524, 11)
y_train 크기:  (4524,)
X_val 크기:  (970, 11)
y_val 크기:  (970,)
X_test 크기:  (969, 11)
y_test 크기:  (969,)


#### T-SNE

In [19]:
from sklearn.manifold import TSNE
'''
##T-SNE 2차원
tsne = TSNE(n_components=2, random_state=42)
X_reduced = tsne.fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)

'''
'''
##T-SNE 3차원
tsne = TSNE(n_components=3, random_state=42)
X_reduced = tsne.fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
#--------------
'''
##T-SNE 2차원
tsne = TSNE(n_components=2, random_state=42)
X_reduced = tsne.fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)


'''
##T-SNE 3차원
tsne = TSNE(n_components=3, random_state=42)
X_reduced = tsne.fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)


print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_val 크기: ", X_val.shape
print "y_val 크기: ", y_val.shape
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (4524, 3)
y_train 크기:  (4524,)
X_val 크기:  (970, 3)
y_val 크기:  (970,)
X_test 크기:  (969, 3)
y_test 크기:  (969,)


#### PCA 

In [8]:
from sklearn.decomposition import PCA
'''
##PCA 2차원
X_pca_reduced = PCA(n_components=2, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_pca_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)

'''
'''
##PCA 3차원
X_pca_reduced = PCA(n_components=3, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_pca_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
'''
##PCA 2차원
X_pca_reduced = PCA(n_components=2, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_pca_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
##PCA 3차원
X_pca_reduced = PCA(n_components=3, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_pca_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)

print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_val 크기: ", X_val.shape
print "y_val 크기: ", y_val.shape
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (4524, 3)
y_train 크기:  (4524,)
X_val 크기:  (970, 3)
y_val 크기:  (970,)
X_test 크기:  (969, 3)
y_test 크기:  (969,)


#### LLE 

In [33]:
from sklearn.manifold import LocallyLinearEmbedding
##LLE 2차원
'''
X_lle_reduced = LocallyLinearEmbedding(n_components=2, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_lle_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
'''
##LLE 3차원
X_lle_reduced = LocallyLinearEmbedding(n_components=3, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_lle_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
##---
'''
##LLE 2차원

X_lle_reduced = LocallyLinearEmbedding(n_components=2, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_lle_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)

'''
##LLE 3차원
X_lle_reduced = LocallyLinearEmbedding(n_components=3, random_state=42).fit_transform(c_data)
X_train, X_test, y_train, y_test = train_test_split(X_lle_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)



print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_val 크기: ", X_val.shape
print "y_val 크기: ", y_val.shape
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (4524, 3)
y_train 크기:  (4524,)
X_val 크기:  (970, 3)
y_val 크기:  (970,)
X_test 크기:  (969, 3)
y_test 크기:  (969,)


#### LDA 

In [57]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
'''
##LDA 2차원
X_lda_reduced = LinearDiscriminantAnalysis(n_components=2).fit_transform(c_data, data['quality'])
X_train, X_test, y_train, y_test = train_test_split(X_lda_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)

'''
'''
##LDA 3차원
X_lda_reduced = LinearDiscriminantAnalysis(n_components=3).fit_transform(c_data, data['quality'])
X_train, X_test, y_train, y_test = train_test_split(X_lda_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
'''
##LDA 6차원
X_lda_reduced = LinearDiscriminantAnalysis(n_components=6).fit_transform(c_data, data['quality'])
X_train, X_test, y_train, y_test = train_test_split(X_lda_reduced, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''

##---
'''
##LDA 2차원
X_lda_reduced = LinearDiscriminantAnalysis(n_components=2).fit_transform(c_data, data['binaryQuality'])
X_train, X_test, y_train, y_test = train_test_split(X_lda_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)


'''
##LDA 3차원
X_lda_reduced = LinearDiscriminantAnalysis(n_components=3).fit_transform(c_data, data['binaryQuality'])
X_train, X_test, y_train, y_test = train_test_split(X_lda_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)

'''
##LDA 6차원
X_lda_reduced = LinearDiscriminantAnalysis(n_components=6).fit_transform(c_data, data['quality'])
X_train, X_test, y_train, y_test = train_test_split(X_lda_reduced, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''

print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_val 크기: ", X_val.shape
print "y_val 크기: ", y_val.shape
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (4524, 1)
y_train 크기:  (4524,)
X_val 크기:  (970, 1)
y_val 크기:  (970,)
X_test 크기:  (969, 1)
y_test 크기:  (969,)


---

## SVM

In [89]:
from sklearn.svm import SVC, LinearSVC

## SVM 분류기

OvA를 사용하여 분류

In [90]:
from sklearn.svm import LinearSVR
lin_clf = LinearSVC(max_iter=100000, random_state=42)
lin_clf.fit(X_train, y_train)

/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

#### 훈련 set에 대한 예측을 만들어 정확도를 측정

In [91]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.7263483642793988

### quality로 했을 때
- 일반: 0.5347038019451813


- T-SNE 2차원: 
- T-SNE 3차원: 0.4398762157382847


- PCA 2차원: 0.4394341290893015
- PCA 3차원: 0.4365605658709107


- LLE 2차원: 0.4328028293545535
- LLE 3차원:0.4334659593280283


- LDA 2차원:0.5327144120247569
- LDA 3차원:0.5327144120247569
- LDA 6차원:0.5360300618921309

### 이진분류로 했을 때
- 일반: 0.7263483642793988


- T-SNE 2차원: 0.6262157382847038
- T-SNE 3차원: 0.6328470380194519


- PCA 2차원: 0.6350574712643678 
- PCA 3차원: 0.6324049513704686


- LLE 2차원: 0.632183908045977
- LLE 3차원:0.6317418213969939


- LDA 2차원:0.7413793103448276
- LDA 3차원:0.7413793103448276
- LDA 6차원: 0.7409372236958444

#### data의 scale 조정

In [92]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_val_scaled = scaler.transform(X_val.astype(np.float32))

In [93]:
lin_clf = LinearSVC(max_iter=100000, random_state=42)
lin_clf.fit(X_train_scaled, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [94]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.7427055702917772

### quality로 했을 때
- 일반: 0.5340406719717065


- T-SNE 2차원: 0.4321396993810787
- T-SNE 3차원: 0.4398762157382847


- PCA 2차원: 0.4378868258178603
- PCA 3차원: 0.4363395225464191


- LLE 2차원: 0.43567639257294427
- LLE 3차원: 0.4376657824933687


- LDA 2차원: 0.5327144120247569
- LDA 3차원: 0.5327144120247569
- LDA 6차원: 0.5360300618921309

### 이진분류로 했을 때
- 일반: 0.7427055702917772


- T-SNE 2차원: 0.6262157382847038
- T-SNE 3차원: 0.6328470380194519


- PCA 2차원: 0.6359416445623343
- PCA 3차원: 0.6339522546419099


- LLE 2차원: 0.6317418213969939
- LLE 3차원:0.632183908045977


- LDA 2차원:0.7413793103448276
- LDA 3차원:0.7413793103448276
- LDA 6차원: 0.7409372236958444

#### RBF 커널 적용

In [95]:
svm_clf = SVC(gamma='auto', decision_function_shape="ovr")
svm_clf.fit(X_train_scaled[:4847], y_train[:4847]) ##4847개 data로 모델 훈련

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [96]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.8006189213085765

### quality로 했을 때
- 일반: 0.6080901856763926


- T-SNE 2차원: 0.44297082228116713
- T-SNE 3차원: 0.4573386383731211


- PCA 2차원: 0.4493810786914235
- PCA 3차원: 0.45755968169761274


- LLE 2차원: 0.4496021220159151
- LLE 3차원: 0.4535809018567639


- LDA 2차원:0.5501768346595933
- LDA 3차원:0.5534924845269673
- LDA 6차원:0.5864279398762158

### 이진분류로 했을 때
- 일반: 0.8006189213085765


- T-SNE 2차원: 0.6315207780725022
- T-SNE 3차원: 0.6522988505747126


- PCA 2차원: 0.6478779840848806
- PCA 3차원: 0.6538461538461539


- LLE 2차원: 0.6412466843501327
- LLE 3차원:0.6410256410256411


- LDA 2차원:0.7393899204244032
- LDA 3차원:0.7393899204244032
- LDA 6차원:0.7769672855879752

#### 교차 검증 이용한 랜덤 서치로 hyper parameter 튜닝

In [97]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, cv=3, n_iter=10, verbose=2, n_jobs=-1)
rnd_search_cv.fit(X_train_scaled, y_train)

rnd_search_cv.best_estimator_
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)
rnd_search_cv.best_score_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.3s finished


0.7734305923961097

In [98]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.7961980548187445

### quality로 했을 때
- 일반: 0.6987179487179487


- T-SNE 2차원: 0.43258178603006187
- T-SNE 3차원: 0.4316976127320955


- PCA 2차원: 0.4458443854995579
- PCA 3차원: 0.4465075154730327


- LLE 2차원: 0.4442970822281167
- LLE 3차원: 0.4498231653404067


- LDA 2차원:0.5446507515473032
- LDA 3차원:0.5517241379310345
- LDA 6차원:0.5820070733863837

### 이진분류로 했을 때
- 일반: 0.8156498673740054


- T-SNE 2차원: 0.6315207780725022
- T-SNE 3차원: 0.6315207780725022


- PCA 2차원: 0.6361626878868258
- PCA 3차원: 0.6472148541114059


- LLE 2차원: 0.6381520778072503
- LLE 3차원:0.637709991158267


- LDA 2차원:0.7411582670203359
- LDA 3차원:0.7431476569407603
- LDA 6차원:0.7765251989389921

In [99]:
##val
y_pred = rnd_search_cv.best_estimator_.predict(X_val_scaled)
accuracy_score(y_val, y_pred)

0.765979381443299

### quality로 했을 때
#### - 일반: 0.6010309278350515


- T-SNE 2차원: 0.4463917525773196
- T-SNE 3차원: 0.44536082474226807


- PCA 2차원: 0.4556701030927835
- PCA 3차원: 0.45979381443298967


- LLE 2차원: 0.45463917525773195
- LLE 3차원: 0.45257731958762887


- LDA 2차원: 0.5381443298969072
- LDA 3차원:0.5422680412371134
- LDA 6차원:0.545360824742268

### 이진분류로 했을 때
#### - 일반: 0.7731958762886598


- T-SNE 2차원: 0.6412371134020619
- T-SNE 3차원: 0.6412371134020619


- PCA 2차원: 0.6484536082474227
- PCA 3차원: 0.6494845360824743


- LLE 2차원: 0.6443298969072165
- LLE 3차원:0.6422680412371135


- LDA 2차원:0.7350515463917526
- LDA 3차원:0.7360824742268042
- LDA 6차원:0.7556701030927835

In [87]:
## 가장 높게 나온 것에 대해서 수행
X_test_scaled = scaler.fit_transform(X_test.astype(np.float32))
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.5717234262125903

In [100]:
## 가장 높게 나온 것에 대해서 수행
X_test_scaled = scaler.fit_transform(X_test.astype(np.float32))
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.7760577915376677